In [1]:
from twikit import Client
import json
import pandas as pd
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as bs
import re
from lxml.html import fromstring
import minsearch
import ast

/Users/vitaly/Documents/Github_personal/llm-zoomcamp/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
from twikit import Client
import json
import pandas as pd

client = Client('en-US')

## You can comment this `login`` part out after the first time you run the script (and you have the `cookies.json`` file)
client.login(
    auth_info_1='',
    password='',
)

client.save_cookies('cookies.json')
client.load_cookies(path='cookies.json')

In [10]:
pd.DataFrame(columns=['full_text', 'entities'], index=[0]).to_csv('tweets.csv', index=False)

In [25]:
tweets_type = "2024 Olympic Men's Basketball articles"
tweets = client.search_tweet(tweets_type, 'Media', count=50)

tweets_to_store = []

for tweet in tweets:
    for url in tweet.urls:
        tweets_to_store.append({
            # 'created_at': tweet.created_at,
            # 'favorite_count': tweet.favorite_count,
            'full_text': tweet.text,
            'entities': url,
        })

# # We can make the data into a pandas dataframe and store it as a CSV file
df = pd.DataFrame(tweets_to_store)
df.to_csv('tweets.csv', index=False, mode='a', header=False)


In [24]:
tweets

[<Tweet id="1817681567470502219">, <Tweet id="1823415379797913684">, <Tweet id="1823178156012122489">, <Tweet id="1823531272632729854">, <Tweet id="1824993003917680959">, <Tweet id="1818006556455354385">, <Tweet id="1823179820273991762">, <Tweet id="1814812470076211657">, <Tweet id="1820722510318743846">, <Tweet id="1721584480249426414">, <Tweet id="1823494957426004405">, <Tweet id="1781323128360022405">, <Tweet id="1816811125029196119">, <Tweet id="1821901921306714351">, <Tweet id="1817366865188098434">, <Tweet id="1823379077970989487">, <Tweet id="1814333643374075981">, <Tweet id="1814773928910348735">, <Tweet id="1749926809108632016">, <Tweet id="1809965643388850465">]

In [26]:
len(df)

15

In [27]:
df = pd.read_csv('tweets.csv')
df

,full_text,entities
0,NaN,NaN
1,@CBC please stop writing articles about Canada...,"{'display_url': 'cbc.ca/sports/basketb…', 'exp..."
2,New Articles in Feed: Olympic Shooting\nRussia...,{'display_url': 'feeddistiller.com/blogs/Olymp...
3,New Articles in Feed: Olympic Shooting\nRussia...,"{'display_url': 'tinyurl.com/FDReader', 'expan..."
4,The U.S. men’s basketball team began its quest...,"{'display_url': 'vogue.com/article/us-men…', '..."
5,Night night! Steph Curry put @USABasketball on...,{'display_url': 'link.thesportsletter.com/view...
6,It’s electric. It’s fast-paced. It’s 3x3 hoops...,"{'display_url': 'slam.ly/usab-3s', 'expanded_u..."
7,"It wasn’t pretty, but the @USABasketball women...",{'display_url': 'link.thesportsletter.com/view...
8,"Paris Olympics, Aug. 6: Live updates and resul...","{'display_url': 'NBA.com', 'expanded_url': 'ht..."
9,"Paris Olympics, Aug. 6: Live updates and resul...","{'display_url': 'NBA.com', 'expanded_url': 'ht..."


In [180]:
# print(json.dumps(tweets_to_store, indent=4))

In [28]:
urls_list = []
for i in range(1, len(df)):
    url = ast.literal_eval(df.loc[i, 'entities'])
    urls_list.append(url['expanded_url'])
urls_list = list( dict.fromkeys(urls_list) )
urls_list

['https://www.cbc.ca/sports/basketball/canada-fiba-world-cup-draw-1.6827012',
 'https://www.feeddistiller.com/blogs/Olympic%20Shooting/feed.html',
 'https://tinyurl.com/FDReader',
 'https://www.vogue.com/article/us-mens-basketball-tips-off-in-paris?utm_source=twitter&utm_medium=social&utm_campaign=dhtwitter&utm_content=app.dashhudson.com%2Fvoguemagazine%2Flibrary%2Fmedia%2F436500605',
 'https://link.thesportsletter.com/view/6009f85950576903ee4927a5lnnm0.mhk/89fcf3be',
 'http://slam.ly/usab-3s',
 'http://NBA.com',
 'http://dlvr.it/TBYGpP',
 'https://www.youtube.com/watch?v=rW5k6MktLsY',
 'https://www.sportsbusinessjournal.com/Articles/2024/08/13/fanatics-olympic-sales-record?publicationSource=morningbuzz&issue=bdb08c9221c346758bb55448ddecb9b3',
 'https://digitaledition.pilotonline.com/html5/mobile/production/default.aspx?edid=f0e64c21-5834-4a32-91a4-af6a59cacd1d',
 'https://ow.ly/t1OQ50SUEko',
 'https://www.theepochtimes.com/sports/veteran-star-laden-us-basketball-squad-will-have-to-dig

In [29]:
sites = ['youtube','instagram','twitter','youtu','shorts','apple','inst','twit','amazo','t.co']
wrongsiteID = [index for index,url in enumerate(urls_list) if any(str(url).lower().__contains__(k) for k in sites)]
urls_list = [url for url in urls_list if url not in wrongsiteID]

In [30]:
articles = {"URL": [],"URL2": [],"Title": [],"Content": []}
counter = 0
checkPoints = 100
for i in tqdm(range(len(urls_list))):
    url = urls_list[i]
    try:
        html = requests.get(url, timeout=(20,20))
        if(html.status_code!=200):
            print("skip",html.status_code,url)
            counter += 1
            continue
        url2 = html.url
        html = html.text
        soup = bs(html, 'html.parser').get_text()
        result = re.sub(r'[\t\r\n]', '', soup)
        tree = fromstring(html)
        title = tree.findtext('.//title')
        articles["URL"].append(url)
        articles["URL2"].append(url2)
        articles['Content'].append(result)
        articles['Title'].append(title)
        print(counter,url2,title)
        if (counter % checkPoints == 0):
            pd.DataFrame(articles).to_csv("articles.csv", index=False)
    except Exception as e:
        print(url, e)
        continue

pd.DataFrame(articles).to_csv("articles.csv", index=False)

 12%|█▎        | 2/16 [00:01<00:11,  1.19it/s]

0 https://www.cbc.ca/sports/basketball/canada-fiba-world-cup-draw-1.6827012 Canadian men's basketball team drawn into group with France at upcoming FIBA World Cup | CBC Sports
https://www.feeddistiller.com/blogs/Olympic%20Shooting/feed.html Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.


 19%|█▉        | 3/16 [00:02<00:08,  1.57it/s]

0 https://www.feeddistiller.com/fdreader.html Feed Distiller - For Blog Readers


 25%|██▌       | 4/16 [00:02<00:06,  1.99it/s]

...

0 https://www.sportsbusinessjournal.com/Articles/2024/08/13/fanatics-olympic-sales-record?publicationSource=morningbuzz&issue=bdb08c9221c346758bb55448ddecb9b3 Fanatics sets new record for Olympic merch sales


 94%|█████████▍| 15/16 [00:15<00:00,  1.06it/s]

0 https://www.spokesman.com/stories/2024/jan/23/former-gonzaga-star-chet-holmgren-named-to-team-us/ Former Gonzaga star Chet Holmgren named to Team USA player pool for Summer Olympics | The Spokesman-Review


100%|██████████| 16/16 [00:19<00:00,  1.20s/it]

0 https://www.nytimes.com/athletic/5621174/2024/07/07/team-usa-basketball-olympics-training-camp-day-1/?utm_medium=social&utm_campaign=twhq&source=twitterhq Inside Day 1 of Team USA’s Olympic training camp: ‘Greatness in the room’ - The Athletic


In [41]:
art_df = pd.read_csv("articles.csv")
art_df

,URL,URL2,Title,Content
0,https://www.cbc.ca/sports/basketball/canada-fi...,https://www.cbc.ca/sports/basketball/canada-fi...,Canadian men's basketball team drawn into grou...,Canadian men's basketball team drawn into grou...
1,https://tinyurl.com/FDReader,https://www.feeddistiller.com/fdreader.html,Feed Distiller - For Blog Readers,Feed Distiller - For Blog ReadersBelieve-it-or...
2,https://www.vogue.com/article/us-mens-basketba...,https://www.vogue.com/article/us-mens-basketba...,The U.S. Men’s Basketball Team—Seeking Their F...,The U.S. Men’s Basketball Team—Seeking Their F...
3,https://link.thesportsletter.com/view/6009f859...,https://link.thesportsletter.com/view/6009f859...,NaN,U.S. Squads Close Out Olympics With Golden Hau...
4,http://slam.ly/usab-3s,https://www.slamonline.com/olympics/usa-mens-3...,USA Basketball's 3x3 Men's Team Arrives in Par...,USA Basketball's 3x3 Men's Team Arrives in Par...
5,http://NBA.com,https://www.nba.com/,The official site of the NBA for the latest NB...,The official site of the NBA for the latest NB...
6,http://dlvr.it/TBYGpP,https://news.google.com/rss/articles/CBMiZ0FVX...,Google News,Google News
7,https://www.youtube.com/watch?v=rW5k6MktLsY,https://www.youtube.com/watch?v=rW5k6MktLsY,How we qualified for Paris // USA Basketball 3...,How we qualified for Paris // USA Basketball 3...
8,https://www.sportsbusinessjournal.com/Articles...,https://www.sportsbusinessjournal.com/Articles...,Fanatics sets new record for Olympic merch sales,Fanatics sets new record for Olympic merch sa...
9,https://digitaledition.pilotonline.com/html5/m...,https://digitaledition.pilotonline.com/html5/m...,The Virginian-Pilot,The Virginian-Pilot


In [39]:
documents = []
for i in range(len(art_df)):
    documents.append({
        'text': art_df.loc[i, 'Content'],
        'title': art_df.loc[i, 'Title'],
        'tweets_type': tweets_type,
    })

In [40]:
documents[4]

{'text': "USA Basketball's 3x3 Men's Team Arrives in Paris Looking for GoldSkip to contentSUBSCRIBESHOPSearch for: NewsNBAWNBACollege & High SchoolFeaturesGoodsChannelsCoversSubscribe AdvertiseDNANewsletterFEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESRefined Elegance: Paolo Banchero Covers SLAM KICKS 27 with the Air Jordan 39By Jared EbanksAugust 16, 2024 Extreme Fandom: Uncovering the Mystery Behind StatMuse and the Legion of Muse Accounts That Just Keep AppearingBy Jared EbanksAugust 8, 2024 Naz Reid Explains His Cult Following, Winning Sixth Man of the Year and Building a Newfound Culture in MinnesotaBy Jared EbanksAugust 7, 2024FEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESFEATURED STORIESRefined Elegance: Paolo Banchero Covers SLAM KICKS 27 with the Air Jordan 39By Jared EbanksAugust 16, 2024 From Undiscovered to Unrivaled, AJ Storr Has His Sights Set on the League After Transferring to KansasBy B

In [93]:
index = minsearch.Index(
    text_fields=["text", "title"],
    keyword_fields=["tweets_type"]
)

In [94]:
index.fit(documents)